In [32]:
import pandas as pd
import numpy as np

In [2]:
df_orders = pd.read_csv('data/instacart-market-basket-analysis/orders.csv')
df_products = pd.read_csv('data/instacart-market-basket-analysis/products.csv')
df_aisles = pd.read_csv('data/instacart-market-basket-analysis/aisles.csv')
df_departments = pd.read_csv('data/instacart-market-basket-analysis/departments.csv')
df_order_products_prior = pd.read_csv('data/instacart-market-basket-analysis/order_products__prior.csv')
df_order_products_train = pd.read_csv('data/instacart-market-basket-analysis/order_products__train.csv')

In [3]:
df_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [12]:
df_order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [19]:
df_combine = df_orders.drop(columns=["eval_set", "order_number"]).join(
    df_order_products_prior.set_index("order_id"), on='order_id', how='inner')
df_combine.head()

,order_id,user_id,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,2,8,NaN,196,1,0
0,2539329,1,2,8,NaN,14084,2,0
0,2539329,1,2,8,NaN,12427,3,0
0,2539329,1,2,8,NaN,26088,4,0
0,2539329,1,2,8,NaN,26405,5,0


In [23]:
df_edge = df_combine.groupby(["user_id", "product_id"]).agg({"order_id": "count"}).reset_index().rename(columns={"order_id": "count"})
df_edge.head()

,user_id,product_id,count
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [24]:
df_edge["count"].value_counts()

1     7982695
2     2204600
3     1029149
4      579462
5      364763
       ...   
95          7
99          5
98          3
96          3
97          3
Name: count, Length: 99, dtype: int64

In [26]:
df_product_features = df_edge.groupby("product_id").agg({"count": "sum", "user_id":"nunique"}).sort_values("count", ascending=False)
df_product_features

,count,user_id
product_id,,
24852,472565,73956
13176,379450,63537
21137,264683,58838
21903,241921,55037
47209,213584,43453
...,...,...
45893,1,1
13397,1,1
30451,1,1


In [38]:
df_product_features["avg_count"] = df_product_features["count"] / df_product_features["user_id"]


In [41]:
df_edge.to_parquet("data/edges.parquet")

In [42]:
df_combine.to_parquet("data/combine.parquet")